In [ ]:
%autosave 60

Autosaving every 60 seconds


Intro

1) **Как обработали данные**

Я не менял предобработку, которая была у нас в бейзлайне. Так что текст был обратотан следующим образом: текст во всех столбцах был объединен в один столбец, дальше текст был разбит на отдельные токены, а также были удалены все символы не буквы и буквы были приведены к нижнему регистру. Дальше была проведена лемматизация с помощью pymorphy3 и удалены стоп слова с помощью библиотеки nltk, а также добавление своих слов. Итоговый текст собирался обратно и это были приведено для тестовых, валидационных и тренировочных данных.

2) **Какой алгоритм вы выбрали**

Я решил использовать для решения этой задачи следующие подходы:

a) Взятть BiLSTM c семинара и постараться доработать его.

б) Взять различные модели типа Bert для русского языка, выделить c помощь них признаки из текста, а уже после использовать линейные слои для получения ответа.

Итог: Лучший результат дал улучшение LSTM, а с помощью bert модели + линейных слоев не получилось получить результат лучше.


3) **Какие особенности использовали при его обучении**

а) Редкие слова, которые встречались меньше 3 раз убирались.

б) Для инцилизации эмбединов у слов использовались fastText.

в) Использовался scheduler для изменения lr при обучении.

4) **Как проводили валидацию алгоритма**

а) При обучении модели использовал разделение на 3 датасета: тренировочный, тестовый, валидационный. Где на тренировчном модель обучалась, на валидационном получал оценку модели, а тестовый использовался для сабмита на kaggle.




Outro:

**Идеи улучшения:**

1) Лучше поработать с трансформерами, так как не получилось с помощью них получить хороший результат, а также использовать ансамбли нескольких моделей для получения предсказаний. Мои попытки работы с трансформерами приведены в отдельном ноутбуке.

2) Лушче поработать с самой предобработкой текста, удалить лишние слова, которые часто поялвяются и не несут особой смысловой нагрузки.

3)* К сожалению (, не удалось много поработать над этим соревнованием, так как сейчас активно готовлюсь к магистратуре и решаю встпупительные экзамены в ВУЗы. Но постараюсь позже улучшить свои результаты.


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# pip install -q razdel pymorphy3 pymorphy3-dicts-ru torch transformers datasets scikit-learn wordcloud graphviz seaborn pandas numpy

In [ ]:
# pip install --upgrade ipywidgets

In [ ]:
# pip install 'accelerate>=0.26.0'

In [ ]:
import os


homebrew_path_m_series = '/opt/homebrew/bin'

if os.path.exists(homebrew_path_m_series):
    os.environ["PATH"] += os.pathsep + homebrew_path_m_series

In [ ]:
!unzip teta-nn-1-2025.zip

Archive:  teta-nn-1-2025.zip
  inflating: sample_submition.csv    
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
pip install razdel

In [ ]:
pip install pymorphy3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 96.0 MB/s eta 0:00:00


In [ ]:
pip install WordCloud

In [ ]:
pip install datasets

In [ ]:
pip install graphviz

In [ ]:
# --- Системные и общие ---
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import warnings
from collections import Counter
warnings.filterwarnings('ignore')

# --- SKlearn ---
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

# --- NLP-инструменты ---
import nltk
from nltk.corpus import stopwords
from razdel import tokenize as razdel_tokenize
import pymorphy3
from wordcloud import WordCloud

# --- PyTorch & Transformers ---
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForSequenceClassification
from datasets import Dataset as HFDataset

# --- Визуализация ---
from graphviz import Digraph

# --- Константы и настройка среды ---
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)

# Определяем устройство: MPS (Apple Silicon GPU), CUDA или CPU
if torch.backends.mps.is_available():
    device = torch.device("mps")
    torch.mps.manual_seed(RANDOM_STATE)
elif torch.cuda.is_available():
    device = torch.device("cuda")
    torch.cuda.manual_seed_all(RANDOM_STATE)
else:
    device = torch.device("cpu")

print(f"✅ Используемое устройство: {device}")

# --- Инициализация лидерборда ---
leaderboard = pd.DataFrame(columns=['Метод', 'R2_Score'])

✅ Используемое устройство: cuda


In [ ]:
full_train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')



# Разбиваем исходный train на обучающую и валидационную выборки
train_df, val_df = train_test_split(
    full_train_df,
    test_size=0.25,
    random_state=RANDOM_STATE
)

# Сбрасываем индексы для удобства
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

# Создаем единое текстовое поле для анализа
text_cols = ['title', 'location', 'company', 'skills', 'description']
for df in [train_df, val_df, test_df]:
    # Заполняем пропуски пустыми строками, чтобы избежать ошибок
    df[text_cols] = df[text_cols].fillna('')
    # Объединяем текстовые колонки через разделитель
    df['full_text'] = df[text_cols].agg(' | '.join, axis=1)

print("Пример данных из train:")
display(train_df[['full_text', 'log_salary_from']].head(3))

Пример данных из train:


,full_text,log_salary_from
0,QA Engineer | Москва (м. Сколково / м. Крылатс...,5.298317
1,Senior Backend QA engineer | Санкт-Петербург |...,5.010635
2,Python разработчик (Django) | Москва | Hammer ...,3.912023


## Предобработка текста



In [ ]:
# Загружаем стоп-слова
nltk.download('stopwords', quiet=True)
stop_words = set(stopwords.words('russian'))
# Добавим специфичные для вакансий стоп-слова
stop_words.update(['наш', 'компания', 'команда', 'работа', 'вакансия', 'искать'])

# Инициализируем лемматизатор pymorphy3
morph = pymorphy3.MorphAnalyzer()

def preprocess_text(text: str) -> str:
    """Полный цикл предобработки текста с использованием pymorphy3."""
    # 1. Токенизация
    tokens = [token.text for token in razdel_tokenize(text)]
    # 2. Приведение к нижнему регистру и удаление не-буквенных токенов
    lower_tokens = [token.lower() for token in tokens if token.isalpha()]
    # 3. Лемматизация и удаление стоп-слов
    lemmas = [morph.parse(token)[0].normal_form for token in lower_tokens]
    cleaned_lemmas = [lemma for lemma in lemmas if lemma not in stop_words]
    return " ".join(cleaned_lemmas)

# Демонстрируем работу на одном примере
sample_text = train_df['full_text'].iloc[0]
print(f"ИСХОДНЫЙ ТЕКСТ: '{sample_text[:200]}...'")
print(f"ОБРАБОТАННЫЙ ТЕКСТ: '{preprocess_text(sample_text)}'")

# Применяем ко всем данным
tqdm.pandas(desc="Preprocessing Train")
train_df['processed_text'] = train_df['full_text'].progress_apply(preprocess_text)
tqdm.pandas(desc="Preprocessing Val")
val_df['processed_text'] = val_df['full_text'].progress_apply(preprocess_text)

ИСХОДНЫЙ ТЕКСТ: 'QA Engineer | Москва (м. Сколково / м. Крылатское) | getmatch agency | Python, Java, Go, C# | Наша компания сохраняет атмосферу стартапа, обеспечивая динамичность и стремление к результатам в сочетани...'
ОБРАБОТАННЫЙ ТЕКСТ: 'qa engineer москва м сколково м крылатский getmatch agency python java go c сохранять атмосфера стартап обеспечивать динамичность стремление результат сочетание комфорт стабильность корпоративный среда трудиться создание будущее российский мобильный связь ключевой продукт российский оператор базовый станция lte g разработать основа принцип открытый архитектура open ran приглашать талантливый специалист присоединиться просто сотрудник сторонник разделять страсть прогресс инновация предстоять заниматься участвовать процесс контроль качество разный этап жизненный цикл разрабатывать средство автоматизация функциональный нагрузочный тестирование разрабатывать тестовый документация участвовать лабораторный приёмочный испытание также составлять отчётный 

Preprocessing Train:   0%|          | 0/12500 [00:00<?, ?it/s]

Preprocessing Val:   0%|          | 0/4167 [00:00<?, ?it/s]

In [ ]:
tqdm.pandas(desc="Preprocessing test")
test_df['processed_text'] = test_df['full_text'].progress_apply(preprocess_text)

Preprocessing test:   0%|          | 0/5556 [00:00<?, ?it/s]

## LSTM



In [ ]:
train_df = pd.read_csv('tran_df_full.csv')
val_df = pd.read_csv('val_df_full.csv')
test_df = pd.read_csv('test_df_full.csv')

In [ ]:
corpus   = [w for text in train_df['processed_text'] for w in text.split()]
word_cnt = Counter(corpus)
vocab    = [w for w,c in word_cnt.items() if c >= 3]
word_to_idx = {w: i+2 for i,w in enumerate(sorted(vocab))}
word_to_idx['<PAD>'] = 0
word_to_idx['<UNK>'] = 1
idx_to_word = {i:w for w,i in word_to_idx.items()}


In [ ]:
!wget -q https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.vec.gz -O /content/cc.ru.300.vec.gz
!gunzip /content/cc.ru.300.vec.gz
FT_PATH = "/content/cc.ru.300.vec"


In [ ]:
embedding_matrix = np.random.normal(scale=0.05, size=(len(word_to_idx), 300)).astype(np.float32)
with open("/content/cc.ru.300.vec", 'r', encoding='utf‑8', newline='\n', errors='ignore') as f:
    next(f)
    for line in f:
        word, *vec = line.rstrip().split(' ')
        if word in word_to_idx:
            embedding_matrix[word_to_idx[word]] = np.array(vec, dtype=np.float32)

embedding_weights = torch.tensor(embedding_matrix)

In [ ]:
class VacancyDataset(Dataset):
    def __init__(self, df, word_to_idx, max_len=150):
        self.texts   = df['processed_text'].values
        self.targets = df['log_salary_from'].values
        self.word_to_idx = word_to_idx
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text   = self.texts[idx].split()
        tokens = [self.word_to_idx.get(t, 1) for t in text][:self.max_len]
        length = len(tokens)
        tokens += [0] * (self.max_len - length)
        return torch.tensor(tokens,  dtype=torch.long), \
               torch.tensor(length, dtype=torch.long),  \
               torch.tensor(self.targets[idx], dtype=torch.float32)

def collate_fn(batch):
    tokens, lengths, targets = zip(*batch)
    tokens   = torch.stack(tokens)
    lengths  = torch.tensor(lengths)
    targets  = torch.stack(targets)
    return tokens, lengths, targets

train_loader = DataLoader(VacancyDataset(train_df, word_to_idx),
                          batch_size=64, shuffle=True, collate_fn=collate_fn)
val_loader   = DataLoader(VacancyDataset(val_df,   word_to_idx),
                          batch_size=64, shuffle=False, collate_fn=collate_fn)


class BiLSTMRegressor(nn.Module):
    def __init__(self, emb_weights, hid_dim=128, n_layers=2, dropout=0.3):
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(
            emb_weights, freeze=False, padding_idx=0)

        self.lstm = nn.LSTM(
            input_size   = emb_weights.shape[1],
            hidden_size  = hid_dim,
            num_layers   = n_layers,
            batch_first  = True,
            bidirectional= True,
            dropout      = dropout if n_layers > 1 else 0.0)

        self.dropout   = nn.Dropout(dropout)
        self.regressor = nn.Linear(hid_dim*2, 1)

    def forward(self, x, lengths):

        emb = self.embedding(x)
        packed = nn.utils.rnn.pack_padded_sequence(
            emb, lengths.cpu(), batch_first=True, enforce_sorted=False)

        packed_out, _ = self.lstm(packed)
        out, _ = nn.utils.rnn.pad_packed_sequence(
            packed_out, batch_first=True)


        mask = (x != 0).unsqueeze(-1)
        out  = out * mask
        mean = out.sum(1) / mask.sum(1).clamp(min=1)

        pred = self.regressor(self.dropout(mean)).squeeze(1)
        return pred

model = BiLSTMRegressor(embedding_weights, hid_dim=128).to(device)


criterion  = nn.MSELoss()
optimizer  = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler  = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, mode='min', factor=0.5, patience=2, verbose=True)

In [ ]:
NUM_EPOCHS = 100
for epoch in range(1, NUM_EPOCHS+1):
    model.train()
    train_loss = 0
    for tokens, lengths, targets in tqdm(train_loader, desc=f'Epoch {epoch}/{NUM_EPOCHS}'):
        tokens, lengths, targets = tokens.to(device), lengths.to(device), targets.to(device)

        optimizer.zero_grad()
        preds = model(tokens, lengths)
        loss  = criterion(preds, targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for tokens, lengths, targets in val_loader:
            tokens, lengths, targets = tokens.to(device), lengths.to(device), targets.to(device)
            preds = model(tokens, lengths)
            val_loss += criterion(preds, targets).item()

    train_loss /= len(train_loader)
    val_loss   /= len(val_loader)
    scheduler.step(val_loss)

    print(f'  train MSE: {train_loss:.4f} | val MSE: {val_loss:.4f}')

from sklearn.metrics import r2_score

model.eval()
all_preds, all_true = [], []
with torch.no_grad():
    for tokens, lengths, targets in val_loader:
        tokens, lengths = tokens.to(device), lengths.to(device)
        preds = model(tokens, lengths).cpu().numpy()
        all_preds.extend(preds)
        all_true.extend(targets.numpy())

r2 = r2_score(all_true, all_preds)
print(f'r2 score: {r2:.4f}')


Epoch 1/100:   0%|          | 0/196 [00:00<?, ?it/s]

  train MSE: 0.0488 | val MSE: 0.1630


Epoch 2/100:   0%|          | 0/196 [00:00<?, ?it/s]

  train MSE: 0.0486 | val MSE: 0.1634


Epoch 3/100:   0%|          | 0/196 [00:00<?, ?it/s]

...

  train MSE: 0.0475 | val MSE: 0.1637


Epoch 34/100:   0%|          | 0/196 [00:00<?, ?it/s]

  train MSE: 0.0480 | val MSE: 0.1637


Epoch 35/100:   0%|          | 0/196 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
class VacancyTestDataset(Dataset):
    def __init__(self, df, word_to_idx, max_len=150):
        self.texts = df['processed_text'].values
        self.word_to_idx = word_to_idx
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        tokens = [self.word_to_idx.get(t, 1)
                  for t in self.texts[idx].split()][:self.max_len]
        length = len(tokens)
        tokens += [0] * (self.max_len - length)
        return torch.tensor(tokens,  dtype=torch.long), \
               torch.tensor(length, dtype=torch.long)

def collate_fn_test(batch):
    tokens, lengths = zip(*batch)
    return torch.stack(tokens), torch.tensor(lengths)


test_dataset = VacancyTestDataset(test_df, word_to_idx, max_len=150)
test_loader  = DataLoader(test_dataset,
                          batch_size=64,
                          shuffle=False,
                          collate_fn=collate_fn_test)


model.eval()
log_preds = []

with torch.no_grad():
    for tokens, lengths in test_loader:
        tokens, lengths = tokens.to(device), lengths.to(device)
        preds_batch = model(tokens, lengths)          # (batch,)
        log_preds.extend(preds_batch.cpu().numpy())


In [ ]:
sub = pd.read_csv('sample_submition.csv')

In [ ]:
sub['prediction'] = log_preds

In [ ]:
sub.to_csv('sub_lstm.csv', index=False)